## Image 기본조작

In [1]:
import numpy as np
import cv2
import os
import sys
import matplotlib.pyplot as plt

In [2]:
print(cv2.__version__)

4.3.0


### 이미지 읽고 화면 출력( Image Read / Show)

In [5]:
#image file read
path = "images/"
fname= "lenna.bmp"
filepath = path+fname

#이미지 파일 읽기
image = cv2.imread(filepath)

if image is None:
    print('Image load failed!')
    sys.exit()

# image를 Widnows로 출력
cv2.imshow(fname, image)

# 칼라 이미지를 그레이스케일로 변환
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow(fname+"- gray", gray_image)

# ESC 키 입력 시 Windows 닫음
cv2.waitKey(0)
cv2.destroyAllWindows() 

### 이미지 속성(Image Properties)

In [6]:
#칼라 이미지 크기, 유형, 총 픽셀수 확인
print("image.shape = ", image.shape)  #행(높이), 열(폭), 채널수
print("image.shape len = ", len(image.shape) ) 
print("image.dtype = ", image.dtype)
print("image.size = ", image.size)

#이미지 높이, 폭, 채널수 확인
imgHeight, imgWidth, channel = image.shape
print(fname, imgHeight, imgWidth, channel)
s = imgHeight * imgWidth * channel
print("image.size = ", s)

#그레이스케일 이미지 크기, 유형, 총 픽셀수 확인
print("\ngray_image.shape = ", gray_image.shape)  #행(높이), 열(폭), 채널수
print("gray_image.shape len = ", len(gray_image.shape) ) 
print("gray_image.dtype = ", gray_image.dtype)
print("gray_image.size = ", gray_image.size)


image.shape =  (512, 512, 3)
image.shape len =  3
image.dtype =  uint8
image.size =  786432
lenna.bmp 512 512 3
image.size =  786432

gray_image.shape =  (512, 512)
gray_image.shape len =  2
gray_image.dtype =  uint8
gray_image.size =  262144


### pixel 직접 접근

In [7]:
# x, y좌표로 직접 접근하여 칼라 픽셀값 얻기
px = image[100,100]
print("image(100,100) = ", px )

# x, y좌표, 칼라채널로 직접 접근하여 각 색상값 얻기
blue = image[100,100,0]
print( "blue = ", blue )
green = image[100,100,1]
print( "green = ", green )
red = image[100,100,2]
print( "red = ", red )

# x, y좌표로 직접 접근하여  그레이스케일 픽셀값 얻기
px = gray_image[100,100]
print("gray image(100,100) = ", px )

gray_image[100,100] = 0
px = gray_image[100,100]
print("gray image(100,100) = ", px )

image(100,100) =  [ 78  68 178]
blue =  78
green =  68
red =  178
gray image(100,100) =  102
gray image(100,100) =  0


### 칼라 이미지 채널 분리 및 결합 (Splitting and Merging Image Channels)

In [8]:
b,g,r = cv2.split(image)
cv2.imshow("Blue", b)
cv2.imshow("Green", g)
cv2.imshow("Red", r)
newimg = cv2.merge((b,g,r))
cv2.imshow("newImage", newimg)

# ESC 키 입력 시 Windows 닫음
cv2.waitKey(0)
cv2.destroyAllWindows() 


### 이미지 관심 영역 ( Image ROI, regions of images)

In [9]:
#lenna.bmp에서 입 영역 직접 추출
clone = image.copy()  #이미지를 복사
sx = 190        # x 시작좌표
ex = sx + 200    # x 끝좌표
sy = 190        # y 시작좌표
ey = sy + 200    # y 끝좌표
ROI = clone[sy:ey,sx:ex]   #영역 추출
cv2.imshow("ROI", ROI)
# ESC 키 입력 시 Windows 닫음
cv2.waitKey(0)
cv2.destroyAllWindows() 

###  image에서 이미지 영역 일부 추출
### 마우스로 ROI 설정하여 추출

In [23]:
#마우스 이벤트 처리
#왼쪽 버튼을 누르는 위치를 시작점, 버튼을 떼는 위치를 끝점으로 설정
#마우스 이동시 사각형을 그려 영역 표시
isDragging = False
sx, sy, w, h = -1, -1, -1, -1
blue, red = (255, 0, 0), (0, 0, 255)
 
def onMouse(event, x, y, flags, param):
    global isDragging, sx, sy, image
    if event == cv2.EVENT_LBUTTONDOWN:  #왼쪽 버튼을 누르면 드래그 시작, 시작좌표 설정
        isDragging = True
        sx = x
        sy = y
    elif event == cv2.EVENT_MOUSEMOVE:  #왼쪽 버튼을 누른채 이동하면 이미지에 파란색 사각형 그리기
        s = "(" + str(x) + "," + str(y) + ")"
        print(s)
        if isDragging:
            img_draw = image.copy()
            cv2.rectangle(img_draw, (sx, sy), (x, y), blue, 2)
            cv2.imshow('img', img_draw)
        else:
            img_draw = image.copy()
            font = cv2.FONT_HERSHEY_COMPLEX  # normal size serif font
            fontScale = 0.8
            cv2.putText(img_draw, s, (x, y), font, fontScale, blue)
            cv2.imshow('img', img_draw)
    elif event == cv2.EVENT_LBUTTONUP:  #왼쪽 버튼을 떼면 
        if isDragging:
            isDragging = False          #드래그 설정 취소
            w = x - sx                  #현재위치에서 시작위치를 빼서 폭과 높이 계산
            h = y - sy
            if w > 0 and h > 0:
                img_draw = image.copy()
                cv2.rectangle(img_draw, (sx, sy), (x, y), red, 2)  #빨간색 사각형으로 최종 영역 확인
                cv2.imshow('img', img_draw)
                roi = image[sy:sy+h, sx:sx+w]          #관심영역(roi)추출
                cv2.imshow('cropped', roi)
                cv2.moveWindow('cropped', 0, 0)     # 관심 영역의 추출 창의 위치를 (0,0) 화면 왼쪽위로 이동
                cv2.imwrite('images/cropped.png', roi)  #관심영역을 파일로 저장
            else:
                cv2.imshow('img', image)
                print('drag should start from left-top side')

#image를 Widnows로 출력
cv2.imshow('img', image)
#마우스 버튼클릭이벤트 ROI 설정
cv2.setMouseCallback('img', onMouse)
cv2.waitKey()
cv2.destroyAllWindows()

(501,129)
(467,134)
(435,139)
(401,144)
(370,147)
(339,151)
(313,155)
(291,163)
(275,166)
(265,170)
(254,173)
(245,176)
(236,179)
(232,180)
(227,182)
(226,182)
(225,183)
(225,183)
(224,183)
(224,183)
(223,183)
(223,183)
(222,184)
(221,184)
(221,184)
(221,183)
(221,182)
(221,181)
(221,180)
(222,178)
(222,177)
(222,175)
(222,172)
(222,169)
(222,167)
(223,164)
(223,161)
(224,158)
(224,155)
(225,153)
(225,149)
(225,148)
(225,146)
(226,144)
(226,144)
(227,143)
(227,142)
(228,141)
(230,141)
(231,140)
(232,140)
(232,140)
(233,140)
(233,140)
(234,140)
(235,140)
(235,140)
(236,140)
(238,140)
(240,140)
(242,139)
(245,138)
(246,138)
(248,138)
(249,138)
(251,137)
(253,137)
(255,137)
(258,137)
(260,137)
(263,137)
(264,137)
(264,137)
(266,137)
(267,137)
(268,138)
(269,138)
(270,138)
(272,139)
(273,139)
(274,139)
(275,141)
(277,141)
(278,142)
(280,143)
(281,143)
(282,144)
(284,145)
(286,145)
(287,146)
(288,147)
(290,148)
(290,149)
(291,149)
(292,149)
(292,150)
(293,150)
(294,151)
(295,152)
(296,153)


(233,247)
(233,248)
(234,248)
(234,249)
(235,249)
(236,249)
(238,249)
(241,249)
(244,251)
(248,252)
(251,253)
(254,253)
(256,254)
(259,256)
(262,257)
(265,257)
(267,258)
(269,260)
(270,260)
(274,262)
(276,263)
(279,265)
(281,266)
(284,267)
(287,269)
(289,270)
(292,271)
(294,272)
(296,273)
(300,274)
(301,275)
(303,276)
(305,276)
(306,276)
(307,276)
(309,277)
(310,277)
(313,277)
(315,278)
(316,278)
(317,278)
(319,279)
(322,279)
(323,279)
(324,279)
(326,279)
(329,280)
(331,280)
(333,280)
(335,280)
(337,280)
(338,280)
(340,280)
(340,280)
(341,280)
(342,280)
(343,281)
(346,281)
(347,281)
(348,281)
(349,281)
(352,282)
(352,282)
(354,282)
(355,282)
(355,282)
(357,283)
(357,283)
(357,283)
(356,283)
(355,283)
(355,283)
(354,283)
(354,283)
(353,283)
(353,283)
(352,283)
(350,283)
(349,283)
(348,283)
(346,284)
(345,284)
(344,284)
(342,284)
(341,285)
(341,285)
(338,285)
(336,286)
(333,287)
(329,288)
(325,289)
(319,290)
(314,291)
(309,291)
(305,292)
(301,293)
(296,294)
(294,295)
(291,296)
(289,296)


### Image Addition & Blending

In [ ]:
img1 = cv2.imread('images/flower1.png')
img2 = cv2.imread('images/cat.png')

dst1 = cv2.add(img1,img2)

a = 0.7
dst2 = cv2.addWeighted(img1, 1-a, img2, a, 0)

cv2.imshow('img1', img1)
cv2.imshow('img2', img2)
cv2.imshow('add', dst1)
cv2.imshow('addWeighted', dst2)

cv2.waitKey()
cv2.destroyAllWindows()